In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
data = data.drop(['Healthcare_1'], axis=1)
test = test.drop(['Healthcare_1'], axis=1)

In [4]:
data.loc[data.HouseYear == 20052011, 'HouseYear'] = 2008
data.loc[data.HouseYear == 4968, 'HouseYear'] = 1968

In [5]:
data = data[data['Square'] < 300]

In [6]:
data.loc[data['Rooms'] == 0, 'Rooms'] = None
test.loc[test['Rooms'] == 0, 'Rooms'] = None

In [7]:
data.loc[data['Square'] < 15, 'Square'] = None
test.loc[test['Square'] < 15, 'Square'] = None

In [8]:
data.loc[data['LifeSquare'] < 15, 'LifeSquare'] = None
test.loc[test['LifeSquare'] < 15, 'LifeSquare'] = None

In [9]:
data.loc[data['LifeSquare'] > 250, 'LifeSquare'] = None
test.loc[test['LifeSquare'] > 250, 'LifeSquare'] = None

In [10]:
data.loc[data['Rooms'] > 5, 'Rooms'] = None
test.loc[test['Rooms'] > 5, 'Rooms'] = None

In [11]:
data.loc[data['KitchenSquare'] > 200, 'KitchenSquare'] = None
test.loc[test['KitchenSquare'] > 200, 'KitchenSquare'] = None

In [12]:
data = pd.get_dummies(data)
test = pd.get_dummies(test)

In [13]:
imputer = IterativeImputer(random_state=0, n_nearest_features=5)
df = pd.DataFrame(imputer.fit_transform(data))
data = data.fillna(df)